In [ ]:
!pip install TTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from TTS.api import TTS # needs Python 3.11 or older
from ask_lisa import ask_lisa

In [ ]:
base_dir = "../asr_bundestag_clean"
train = base_dir + "/train_nodev"
validate = base_dir + "/train_dev"
test = base_dir + "/test"

In [ ]:
mapping = {}

with open(train + "/wav.scp", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split()
        mapping.update({words[0]: [words[1]]})

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

In [ ]:
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split(maxsplit=1)
        mapping[words[0]].append(words[1])

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

In [ ]:
word_count = {}
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        count = len(line.strip().split()) - 1
        if count in word_count:
            word_count[count] = word_count[count] + 1
        else:
            word_count[count] = 1

counts = sorted(word_count.keys())
frequencies = [word_count[c] for c in counts]

plt.bar(counts, frequencies)
plt.xlabel("Wörter pro Transkript")
plt.ylabel("Häufigkeit")
plt.show()



In [ ]:
filtered_mapping = {}
for key, value in mapping.items():
    count = len(value[1].split())
    if 70 < count < 1000:
        filtered_mapping.update({key: value})

mapping = filtered_mapping

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

print(len(mapping.items()))

In [ ]:
import json
from ask_lisa import ask_lisa
def generate_qa(transcript, difficulty):
    instruction = f'zum Training eines SQA models benötige ich Frage-Antwort-Paare zu eingegebenen Samples aus Transkripten. Die Fragen müssen so gestellt sein, dass sie mit maximal zwei Worten beantwortet werden können und die Antworten auch genauso präzise sind. Gib als Antwort immer nur EIN Frage-Antwort-Paar als JSON-Objekt mit den keys "question" und "answer" aus. Keine anderen Texte oder Infos. Gib keine mehreren Antwortmöglichkeiten aus. Immer genau eine Antwort bzw. ein Wort ist richtig.\nKomplexitätsgrad der Frage und Antworten (bei Antworten aber nicht länger als oben gegebenes Limit, dafür aber tiefgründiger) (Skala 1-3, 3 am komplexesten, 1 am einfachsten): {difficulty}\n'
    qa_res = ask_lisa(instruction, transcript)
    qa = json.loads(qa_res)

    return qa['question'], qa['answer']

generate_qa("cdu csu fraktion frau präsidentin meine sehr geehrten damen und herren liebe kolleginnen und kollegen wo man am meisten drauf erpicht grad das bekommt man meistens nicht liebe kolleginnen und kollegen von den linken und den grünen dieses zitat von wilhelm busch passt perfekt zu ihren anträgen gern erläutere ich warum das so ist sie fordern dass die co kosten allein den vermieterinnen und vermietern aufgebürdet werden sollen und erhoffen sich hierdurch dass die energetische sanierung der gebäude angereizt wird", 3)

# Generating the qa-pairs

In [ ]:
pairs = []
for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    # random number between 1 and 3
    difficulty = random.randint(1, 3)
    print(key)
    qaPair = generate_qa(value[1], difficulty)
    pairs.append({
        'transcript': value[1],
        'qa-pair': qaPair,
        'id': key
    })

# save pairs as json to file
with open("qa_pairs.json", "w", encoding="utf-8") as f:
    json.dump(pairs, f, ensure_ascii=False, indent=4)

In [ ]:
tts = TTS("tts_models/de/thorsten/vits", progress_bar=False)
def to_speech(text):
    tts.tts_to_file(text=text, file_path="output.wav")

In [ ]:
keys = list(mapping.keys())
     
for i in range(1):
    key = random.choice(keys)
    print(mapping[key][1])
    question, answer = generate_qa(mapping[key][1])
    to_speech(question)
    